In [ ]:
import os
import json
import requests


In [ ]:


def ensure_folder_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)  # 自動建立資料夾



In [68]:
prefixs = [ "LoRA_Trigger", "LoRA_noTrigger", "noLoRA_Trigger", "noLoRA_noTrigger"]


In [69]:
def rename_files():
    base_path = "..\\data\\input"
    for p in prefixs:
        folder_path = os.path.join(base_path, p)
        ensure_folder_exists(folder_path)
        counter = 0
        for file_name in os.listdir(folder_path):
            src = os.path.join(folder_path, file_name)
            dst = os.path.join(folder_path, p + "_" + str(counter) + ".png")
            os.rename(src, dst)
            counter += 1


In [70]:
rename_files()

FileExistsError: [WinError 183] 當檔案已存在時，無法建立該檔案。: '..\\data\\input\\LoRA_Trigger\\LoRA_Trigger_10.png' -> '..\\data\\input\\LoRA_Trigger\\LoRA_Trigger_2.png'

In [71]:
# read api from json file
api = {}
with open("anaysis with LoRA api.json", "r") as f:
    api = f.read()
api = json.loads(api)

trigger_prompt = "zzMimikyu, solo, pokemon (creature), no humans, animal focus, mimikyu, "
object_prompt = "a cute rat"

In [72]:
class Config:
    def __init__(self, model, LoRA, input_image, trigger_prompt, object_prompt, file_prefix, total_steps, current_step):
        self.model = model
        self.LoRA = LoRA
        self.input_image = input_image
        self.trigger_prompt = trigger_prompt
        self.object_prompt = object_prompt
        self.file_prefix = file_prefix
        self.total_steps = total_steps
        self.current_step = current_step

    def add_step(self):
        self.current_step += 1

    def set_input_image(self, input_image):
        self.input_image = input_image

config = Config(
    model="dreamshaper_8.safetensors",
    LoRA="Mimikyu_Pokemon_PDXL.safetensors",
    input_image="input_image.png",
    trigger_prompt=trigger_prompt,
    object_prompt=object_prompt,
    file_prefix=prefixs,
    total_steps=20,
    current_step=0
)

In [73]:


def modify_api(api, config, counter = 0):
    api["prompt"]["3"]["inputs"]["ckpt_name"] = config.model
    api["prompt"]["2"]["inputs"]["lora_name"] = config.LoRA
    api["prompt"]["7"]["inputs"]["image"] = config.input_image
    api["prompt"]["4"]["inputs"]["text"] = config.trigger_prompt + config.object_prompt
    api["prompt"]["11"]["inputs"]["filename_prefix"] = f"{config.file_prefix}/{config.object_prompt}/{config.input_image}/{config.current_step}"
    api["prompt"]["15"]["inputs"]["filename_prefix"] = f"{config.file_prefix}/{config.object_prompt}/{config.input_image}/{config.current_step}@{counter}@"
    api["prompt"]["10"]["inputs"]["steps"] = config.total_steps
    api["prompt"]["10"]["inputs"]["denoise"] =  1-config.current_step/ config.total_steps
    api["prompt"]["1"]["inputs"]["start_at_step"] = config.current_step
    api["prompt"]["1"]["inputs"]["end_at_step"] = config.current_step + 1
    return api

In [75]:
import shutil
output_folder = "..\\..\\..\\myComfyUI\\output\\lab\\hspace"
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

In [76]:
import shutil


generate_counter = 1

for prefix in prefixs:
    config.file_prefix = prefix
    object_prompt = "a cute rat"
    config.object_prompt = object_prompt
    for input_image in os.listdir(f"..\\..\\..\\myComfyUI\\input"):
        if prefix not in input_image:
            continue
        print(f"{prefix} {object_prompt} {input_image}")
        config.set_input_image(input_image)
        config.current_step = 0
        for i in range(20-1):
            config.current_step = i
            api = modify_api(api, config, generate_counter)
            generate_counter += 1
            send_request(api)
            print(f"{prefix} {object_prompt} {i} done")

LoRA_Trigger a cute rat LoRA_Trigger_0.png
200
{'prompt_id': '2641f5ad-a417-4b3b-8065-8e62022aba7e', 'number': 2028, 'node_errors': {}}
LoRA_Trigger a cute rat 0 done
200
{'prompt_id': 'c685179a-9489-4e18-ba71-6513577d8e7d', 'number': 2029, 'node_errors': {}}
LoRA_Trigger a cute rat 1 done
200
{'prompt_id': 'b4f0d604-c663-4e3a-bdf9-451fc359521c', 'number': 2030, 'node_errors': {}}
LoRA_Trigger a cute rat 2 done
200
{'prompt_id': '3bbfdab7-6097-46c5-9934-ce6a524283a8', 'number': 2031, 'node_errors': {}}
LoRA_Trigger a cute rat 3 done
200
{'prompt_id': 'd194d65e-9c16-4f40-8965-ebdce967da35', 'number': 2032, 'node_errors': {}}
LoRA_Trigger a cute rat 4 done
200
{'prompt_id': '7307cec1-5092-4643-add0-7b6b4473afa7', 'number': 2033, 'node_errors': {}}
LoRA_Trigger a cute rat 5 done
200
{'prompt_id': '1032ed72-2d5a-4e04-8da6-842b399140f4', 'number': 2034, 'node_errors': {}}
LoRA_Trigger a cute rat 6 done
200
{'prompt_id': 'b3ae58a1-8bb5-411b-b938-fa0fe341d183', 'number': 2035, 'node_errors': 

In [ ]:
import json
import requests
def send_request(api):
    # 3. 設定 API 的 URL 與 headers
    url = 'http://127.0.0.1:8188/prompt'
    headers = {'Content-Type': 'application/json'}

    # 4. 發送 POST request 並傳送修改後的 JSON 資料
    response = requests.post(url, headers=headers, data=json.dumps(api))

    # 5. 輸出回應結果
    print(response.status_code)
    print(response.json())

In [66]:
config2 = Config(
    model="dreamshaper_8.safetensors",
    LoRA="Mimikyu_Pokemon_PDXL.safetensors",
    input_image="noLoRA_Trigger_5.png",
    trigger_prompt=trigger_prompt,
    object_prompt=object_prompt,
    file_prefix=prefixs[0],
    total_steps=20,
    current_step=15
)
send_request(modify_api(api, config2, 18))

200
{'prompt_id': 'e6332703-ab44-4d67-8ab5-5be461ee987c', 'number': 2027, 'node_errors': {}}


In [1]:
hspace_folder = "..\\..\\..\\myComfyUI\\output\\lab\\hspace"
data_folder = "..\\data"

for dir in os.listdir(data_folder):
    print(dir)


NameError: name 'os' is not defined